In [1]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import random
from collections import OrderedDict

torch.set_printoptions(edgeitems=2)

In [2]:
# 定义函数

#折射率
def n(omega):
    if(omega>1e14):
        t=100
        a1=4.582
        a2=0.09921
        a3=0.2109
        a4=-0.02194
        b1=0.00000022971
        b2=0.000000052716
        b3=-0.000000049143
        c=88506.25
        t1=t+273.15
        
        w=(2*np.pi*3e8/omega)*1e6
        c1=w*w-(a3+b3*(t1*t1-c))*(a3+b3*(t1*t1-c))
        c2=a1+b1*(t1*t1-c)+(a2+b2*(t1*t1-c))/c1+a4*w*w
        y=np.sqrt(abs(c2))
    else:
        d1=19.9
        d2=44
        d3=4.533e12*2*np.pi
        d4=0.426e12*2*np.pi
        d5=2.9176e13
        
        e=d1+(d2-d1)*d5**2/(d3**2+1.0j*omega*d4-omega**2)
        y=np.real(np.sqrt(abs(e)))

    return y

def kk(f):
    z=f*n(f)/3e8
    return z

def funct(val):
    G=np.zeros([2,300])
    d=np.zeros([2,100])
        
    d[1,:] = val
        
    for j in range(100):
        d[0,j] = j*3e-6
        
    for k in range(300):
        f_thz=0.1e12+k*0.15e12
        f_nir=3e8/800e-9
        G[0,k]=-kk(2*np.pi*f_nir)+kk(2*np.pi*(f_nir-f_thz))+kk(2*np.pi*f_thz)
        
    for ii in range(300):
        for jj in range(100):
            if jj == 99:
                break
            G[1,ii]=G[1,ii]+d[1,jj]*np.exp(1.0j*G[0,ii]*d[0,jj+1])-np.exp(1.0j*G[0,ii]*d[0,jj])/(1.0j*G[0,ii])
        
    peak = np.max(np.abs(G[1,:]))
    G_au = np.abs(G[1,:])/peak
        
    max_index = G_au.tolist().index(1)
        
    lo = max_index 
    # 前向搜索
    while G_au[lo] > 0.5 and lo > 0:
        lo -= 1
        if G_au[lo] <= 0.5:
            break 

    hi = max_index
    # 后向搜索
    while G_au[hi] > 0.5 and hi < 299:
        hi += 1
        if G_au[hi] <= 0.5:
            break

    if(hi==299 and G_au[hi]>0.5)or(lo==0 and G_au[lo]>0.5):
        bw=0.01
    else:
        bw = hi - lo
    return bw

In [3]:
# 定义拟合网络：将输入的100个变量映射到最后的值：
class FitModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.hidden_linear = nn.Linear(100, 300)
        self.hidden_activation = nn.ReLU()
        self.output_linear = nn.Linear(300, 1)
        
    def forward(self, input):
        hidden_t = self.hidden_linear(input)
        activated_t = self.hidden_activation(hidden_t)
        output_t = self.output_linear(activated_t)
        
        return output_t

# 将定义的模型实例化：
fitNet = FitModel()

# 为实例化后的模型定义优化器
fitOpt = optim.SGD(fitNet.parameters(), lr=1e-4)

In [4]:
# 定义生成网络：生成100个输入值：

# 由于输入值只能为1或-1，所以选取tanh函数，通过对x放大，可以将实数域的值
# 映射到近似1或-1两个值（可以绘制tanh(100x)的图形帮助理解）
# 于是定义如下激活函数：
class MyTanh(nn.Module):
    def __init__(self):
        super(MyTanh, self).__init__()
        
    def forward(self, x):
        x = torch.tanh(100 * x)
        return x 

# 接下来定义生成网络：
class GenModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        # 网络1：得出输入
        self.hidden_linear = nn.Linear(100, 300)
        self.hidden_activation = nn.Tanh()
        self.output_linear = nn.Linear(300, 100)
        self.output_activation = MyTanh()
        
    def forward(self, input):
        hidden_t = self.hidden_linear(input)
        activated_t = self.hidden_activation(hidden_t)
        output_t = self.output_linear(activated_t)
        opa = self.output_activation(output_t)
        
        return opa

genNet = GenModel()
genOpt = optim.SGD(genNet.parameters(), lr=1e-4)

In [5]:
# 训练拟合网络：

# 1. 构造训练集与测试集：
# 输入获取函数
def generateInput(size):
    inputList = []
    
    for i in range(size): 
        inList = []
        for j in range(100):
            item = random.randint(0, 1)
            if item == 0:
                item = -1
            inList.append(item)
        inputList.append(inList)
        
    return inputList

# 输出获取函数
def getOutput(inputList):
    outputList = []
    for i in range(len(inputList)):
        G=np.zeros([2,300])
        d=np.zeros([2,100])
        
        d[1,:] = np.array(inputList)[i,:]
        
        for j in range(100):
            d[0,j] = j*3e-6
        
        for k in range(300):
            f_thz=0.1e12+k*0.15e12
            f_nir=3e8/800e-9
            G[0,k]=-kk(2*np.pi*f_nir)+kk(2*np.pi*(f_nir-f_thz))+kk(2*np.pi*f_thz)
        
        for ii in range(300):
            for jj in range(100):
                if jj == 99:
                    break
                G[1,ii]=G[1,ii]+d[1,jj]*np.exp(1.0j*G[0,ii]*d[0,jj+1])-np.exp(1.0j*G[0,ii]*d[0,jj])/(1.0j*G[0,ii])
        
        peak = np.max(np.abs(G[1,:]))
        G_au = np.abs(G[1,:])/peak
        
        max_index = G_au.tolist().index(1)
        
        lo = max_index 
        # 前向搜索
        while G_au[lo] > 0.5 and lo > 0:
            lo -= 1
            if G_au[lo] <= 0.5:
                break 
        
        hi = max_index
        # 后向搜索
        while G_au[hi] > 0.5 and hi < 299:
            hi += 1
            if G_au[hi] <= 0.5:
                break
        
        if(hi==299 and G_au[hi]>0.5)or(lo==0 and G_au[lo]>0.5):
            bw=-1
        else:
            bw = hi - lo
        outputList.append(bw)
    
    return outputList

In [6]:
# 获取输入数据与输出数据：
inputList = generateInput(50)
outputList = getOutput(inputList)

C:\Users\20241\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:40: ComplexWarning: Casting complex values to real discards the imaginary part


In [7]:
# 对输入数据与输出数据tensor包装，以便输入神经网络进行训练：
inputList = torch.tensor(inputList)
outputList = torch.tensor(outputList).unsqueeze(1)

# 进行训练集与测试集划分：
n_samples = inputList.shape[0]
n_val = int(0.2 * n_samples)
shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

input_train = inputList[train_indices]     # 训练集：输入数据
output_train = outputList[train_indices]   # 训练集：输出数据

input_val = inputList[val_indices]         # 测试集：输入数据
output_val = outputList[val_indices]       # 测试集：输出数据

In [8]:
# 定义拟合网络的训练函数：
def fitTrain(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)
        
        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if epoch == 1 or epoch % 50 == 0:
            print('Epoch {}, Training loss {}, Validation loss {}'.format(
                epoch, float(loss_train), float(loss_val)))

In [9]:
# 进行拟合函数的训练：
fitTrain(
    n_epochs = 200, 
    optimizer = fitOpt,
    model = fitNet,
    loss_fn = nn.MSELoss(),
    t_u_train = input_train.float(),
    t_u_val = input_val.float(), 
    t_c_train = output_train.float(),
    t_c_val = output_val.float())

Epoch 1, Training loss 36.82501220703125, Validation loss 7.651153564453125
Epoch 50, Training loss 30.65987777709961, Validation loss 4.707790851593018
Epoch 100, Training loss 26.146820068359375, Validation loss 2.8960328102111816
Epoch 150, Training loss 22.84134292602539, Validation loss 1.8686186075210571
Epoch 200, Training loss 20.360076904296875, Validation loss 1.3463401794433594


In [10]:
# 拟合网络训练结束后，冻结其全部参数，以免在生成网络的训练过程中被修改：
fitNet.require_grad = False

In [11]:
# 定义生成网络的训练：
# 通过输出的out参数：genNet的参数确实发生了变化，但是变化很小，甚至转为1或-1时就没变化了
def genTrain(n_epochs, optimizer, model):
    val = torch.ones(100)
    out = 0
    for epoch in range(1, n_epochs + 1):
        out = model(val)
        
        # 我们希望fitNet(out)的值越大越好，所以取倒数作为损失函数（乘10为了结果不是特别小）
        loss = 10/fitNet(out)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch == 1 or epoch % 100 == 0:
            print('Epoch {}, loss {}, out {}'.format(
                epoch, float(loss), out))
    return out

In [12]:
# 进行生成网络的训练：
out = genTrain(500, genOpt, genNet)

Epoch 1, loss 4.22904109954834, out tensor([ 1.0000, -0.9996, -1.0000, -0.5706, -0.9994, -1.0000, -1.0000, -1.0000,
         1.0000,  1.0000,  1.0000,  1.0000, -1.0000,  1.0000, -1.0000, -1.0000,
         0.6073, -1.0000, -1.0000,  1.0000,  0.4620, -1.0000, -1.0000, -1.0000,
         1.0000, -0.9970, -1.0000, -1.0000, -1.0000, -1.0000,  1.0000, -1.0000,
         1.0000,  1.0000,  0.9999, -1.0000,  1.0000, -1.0000, -1.0000, -1.0000,
         1.0000,  1.0000,  1.0000, -0.9350, -1.0000, -1.0000,  1.0000, -1.0000,
         1.0000,  0.9998, -1.0000, -1.0000,  1.0000,  1.0000, -1.0000, -1.0000,
         1.0000, -1.0000,  1.0000, -0.8049,  1.0000,  1.0000,  1.0000,  1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -0.7911,  1.0000, -0.9777,
         1.0000, -1.0000,  1.0000,  1.0000, -1.0000,  1.0000, -1.0000,  1.0000,
        -1.0000, -1.0000,  1.0000, -1.0000, -1.0000, -1.0000,  1.0000, -1.0000,
         1.0000, -1.0000, -1.0000, -1.0000, -0.9918,  1.0000,  1.0000, -1.0000,
    

In [13]:
out

tensor([ 1.0000, -1.0000, -1.0000,  0.9999, -0.9998, -1.0000, -1.0000, -1.0000,
         1.0000,  1.0000,  1.0000,  1.0000, -1.0000,  1.0000, -1.0000, -1.0000,
        -1.0000, -1.0000, -1.0000,  1.0000,  0.9994, -1.0000, -1.0000, -1.0000,
         1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -1.0000,  1.0000, -1.0000,
         1.0000,  1.0000,  1.0000, -1.0000,  1.0000, -1.0000, -1.0000, -1.0000,
         1.0000,  1.0000,  1.0000, -0.9998, -1.0000, -1.0000,  1.0000, -1.0000,
         1.0000,  1.0000, -1.0000, -1.0000,  1.0000,  1.0000, -1.0000, -1.0000,
         1.0000, -1.0000,  1.0000, -0.9999,  1.0000,  1.0000,  1.0000,  1.0000,
        -1.0000, -1.0000, -1.0000, -1.0000, -1.0000, -0.9999,  1.0000, -0.9998,
         1.0000, -1.0000,  1.0000,  1.0000, -1.0000,  1.0000, -1.0000,  1.0000,
        -1.0000, -1.0000,  1.0000, -1.0000, -1.0000, -1.0000,  1.0000, -1.0000,
         1.0000, -1.0000, -1.0000, -1.0000, -0.9999,  1.0000,  1.0000, -1.0000,
        -1.0000, -1.0000,  1.0000,  1.00